In [13]:
import time
import pandas as pd
import os

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

: 

In [ ]:
def get_FAQ_data(driver):
    page_len = driver.find_elements(By.CLASS_NAME, "page_link")
    page_len = len(page_links) # 총 FAQ 페이지 수
    
    faq_data = []
    for i in range(page_len):
        time.sleep(2)
        # 한 페이지에 있는 질문 리스트 추출
        branch_search = driver.find_element(By.CLASS_NAME, 'branch_serch')
        branch_list = branch_search.find_elements(By.CLASS_NAME, 'branch_wrap')
        print(f"----------- {i+1}번째 데이터 길이: ", len(branch_list), "-----------")

        for j in range(len(branch_list)):
            # 질문 및 답변 추출
            try:
                btn = driver.find_element(By.CSS_SELECTOR, f'#root > div > div > div.container > div > div.contents > div.branch_serch > div:nth-child({j+2}) > a')
                #driver.execute_script("arguments[0].scrollIntoView(true);", btn)
                time.sleep(1) 
                btn.click()

                time.sleep(3)
                
                question = driver.find_element(By.CSS_SELECTOR, '#root > div > div > div.container > div > div.contents > div.board_detail > div.board_title > p').text
                answer = driver.find_element(By.CSS_SELECTOR, '#root > div > div > div.container > div > div.contents > div.board_detail > div.board_contents.mt20 > p').text
                #
                # print(question)

                # 데이터 저장
                faq_data.append([question, answer])

                # 목록으로 돌아가기
                back_btn = driver.find_element(By.CSS_SELECTOR, '#root > div > div > div.container > div > div.contents > div.btn_group.single > a')
                back_btn.click()
                time.sleep(3)

            except Exception as e:
                    print(f"Error on page {i+1}, question {j+1}: {e}")
                    break
        
        print(f"----------- {i+1} 번째 페이지 크롤링 종료 -----------")
        time.sleep(2)

        if i < page_len - 1:
            next_page_btn = driver.find_element(By.CSS_SELECTOR, f'#root > div > div > div.container > div > div.contents > div.branch_serch > nav > ol > li:nth-child({i+2})')
            next_page_btn.click()
            time.sleep(3)  # 다음 페이지 로드 대기
            
    print(f"----------- FAQ 크롤링 종료 -----------")
    return faq_data
    
    
def safe_data(df):
    result_path = os.path.join(os.getcwd(), 'result_data')
    os.makedirs(result_path, exist_ok=True)
    df.to_csv(os.path.join(result_path, 'faq_df.csv'), index=False, encoding='utf-8')
    
def get_prepare(url):
    # 크롬 드라이버 생성 및 url 접속
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(3)
    
    result = get_FAQ_data(driver)
    faq_df = pd.DataFrame(result, columns=['Question', 'Answer'])
    safe_data(faq_df)

In [7]:
## main.py
faq_url = "https://homepage.skcarrental.com/customer/faq"
get_prepare(faq_url)

36